# Multi-Label Classification

In [2]:
import os
import pandas as pd
from pandas.plotting import autocorrelation_plot
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score

In [110]:
df = pd.read_csv('C:\Repos\DS-3FeaturesClassification\df_data.csv')
df = pd.get_dummies(df,columns=['target'],prefix = ['target'])
x = np.dstack( (np.array(df['x1']) , np.array(df['x2']), np.array(df['x3'])) )
y = np.dstack( (np.array(df['_target'], (np.array(df['target_low']) , np.array(df['target_med']), np.array(df['target_high'])) )

x = x[0,:,:]
y = y[0,:,:]

SyntaxError: invalid syntax (<ipython-input-110-e5665ee49ff4>, line 6)

In [98]:
df['target_low'].value_counts(normalize=True)

1    0.6
0    0.4
Name: target_low, dtype: float64

In [99]:
df['target_med'].value_counts(normalize=True)

0    0.9
1    0.1
Name: target_med, dtype: float64

In [100]:
df['target_low'].value_counts(normalize=True)

0    0.7
1    0.3
Name: target_high, dtype: float64

In [52]:
arr_dict = {0:"low", 1:"med", 2:"high"}
def arr2class(_arr):
    return arr_dict[np.argmax(_arr)]

In [72]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
true_arr = []
for arr in y_test:
    true_arr.append(arr2class(arr))

df_pred = pd.DataFrame()
df_pred['true'] = true_arr

# Escalando dados para deep learning

## Função de ativação e otimizador.

Função de ativação 

Como convertemos os valores negativos em escalas individuais de 0 a 1 para cada feature x1,x2,x3 podemos utilizar melhor as funções de ativação compatíveis com tais valores, no caso a ReLu é apropriada para o seleção de features visto que sua derivada produz uma função degrau que irá desativar ou ativar features para cada label.

### Funções de ativação:
[<img src="https://4.bp.blogspot.com/-4puZ_ZMyoLE/WPk_rwqhKnI/AAAAAAAAAjU/vrNE_Uv54yMLOd_3E83PvgpByf019ufZwCLcB/s400/ActivationFunctions.png">]

[<img src="https://1.bp.blogspot.com/-eT7Sq-cmXSQ/WPk_r6agDnI/AAAAAAAAAjY/uWakDV5UBSw_5L0Sw53XLnogv2ExNOQRwCEw/s400/ActivationFunctionDerivatives.png">]

Otimizador para o treino backpropagation será escolhido Adam, que em comparação a outros otimizadores de reajuste de gradiente, é o que converge mais rápido!

[<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/05/Comparison-of-Adam-to-Other-Optimization-Algorithms-Training-a-Multilayer-Perceptron.png">]


Fonte: Machine Learning Mastery
https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

## Modelagem da rede.  
Podemos modelar inicialmente a rede neural com 9 neuronios na entrada, inferindo que cada 1 das 3 saídas é uma combinação específica de 3 features x1,x2,x3 da entrada. A saída será de 3 neuronios pois teremos uma classificação binária de 3 labels high(1,0,0), low(0,1,0), med(0,0,1)


In [103]:
current_model = 'mlp1'
mlp1 = Sequential()
mlp1.add(Dense(9, input_dim=3, kernel_initializer='he_uniform', activation='tanh')) #tanh para admitir entrada de valores negativos
mlp1.add(Dense(3, activation='relu')) # saida com função degrau
mlp1.compile(loss='binary_crossentropy', optimizer='adam')
mlp1.save_weights(current_model + "_wheigts_initial.h5")

history1 = mlp1.fit(x_train, y_train, verbose=0, epochs=9)
results1 = mlp1.predict(x_test)

test_arr = []
for arr in results1:
    test_arr.append(arr2class(arr))
df_pred['mlp1_predicted_class'] = test_arr
df_pred['mlp1_score'] = np.where(df_pred["true"] == df_pred["mlp1_predicted_class"], True, False)
df_pred['mlp1_score'].describe()

count     2500
unique       2
top       True
freq      1457
Name: mlp1_score, dtype: object

# KNN

Para este problema podemos utilizar o algoritmo popular KNN(K — Nearest Neighbors), pois é um algoritmo de treino supervisionado (e este problema temos o valor supervisionado y = "target" para cada entrada de treinamento)

E as entradas de treino possuem valores quantitativos que podem ser interpretados como distancias euclidianas no espaço entre as features até o valor supervisionado.

No exemplo abaixo, as features x1,x2,x3 poderiam ser intepretadas como Wheelbase, price in thousands e horsepower.


[<img src="https://www.ibm.com/support/knowledgecenter/bs/SSLVMB_24.0.0/spss/images/images_m-r/model_knn_feature_space_cars_02.jpg">]




In [91]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(x_train,y_train)
results2 = knn_clf.predict(x_test)

test_arr = []
for arr in results2:
    test_arr.append(arr2class(arr))
df_pred['knn_predicted_class'] = test_arr
df_pred['knn_score'] = np.where(df_pred["true"] == df_pred["knn_predicted_class"], True, False)
df_pred['knn_score'].describe()

count     2500
unique       2
top       True
freq      1379
Name: knn_score, dtype: object

# COMPARANDO

In [109]:
print("\n\n     MLP \n\n", df_pred['mlp1_score'].value_counts(normalize=True))
print("\n\n     KNN \n\n", df_pred['knn_score'].value_counts(normalize=True))



     MLP 

 True     0.5828
False    0.4172
Name: mlp1_score, dtype: float64


     KNN 

 True     0.5516
False    0.4484
Name: knn_score, dtype: float64


# REDE NEURAL ACERTOU 58,28% E KNN 55,16%